In [1]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image
import torch
from torch import nn

%load_ext autoreload
%autoreload 2

In [2]:
# Getting Data
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "How many cats are there?"

In [3]:
# Initializing Model
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
# prepare inputs
encoding = processor(image, text, return_tensors="pt")

# Initializing Dict for storing activation values
activations = {}


Downloading:   0%|          | 0.00/251 [00:00<?, ?B/s]

c:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\huggingface_hub\file_download.py:125: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/136k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/470M [00:00<?, ?B/s]

In [5]:
encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask'])

In [7]:
# Creating a hook
def create_hook(name, out):
  #print(name)
  def hook(module, in_tensor, out_tensor):
      out[name] = out_tensor
  return hook

# Loop through all layers of the model and choose the ones with an activaton function to register a hook
def get_all_activation_layers(net):
  for name, layer in net._modules.items():
    if name == "vilt":
      l1 = list(list(layer._modules.items())[1][1]._modules.items())[0][1]
      if isinstance(l1, nn.ModuleList):
        for i in range(len(l1)):
          n, l2 = list(list(l1[i]._modules.items())[1][1]._modules.items())[1]
          name_list = n + "_" + str(i)
          l2.register_forward_hook(create_hook(name_list, activations))
      n3, l3 = list(list(layer._modules.items())[3][1]._modules.items())[1]
      l3.register_forward_hook(create_hook(n3, activations))
    else:
      n4, l4 = list(layer._modules.items())[2]
      l4.register_forward_hook(create_hook(n4, activations))

In [8]:
get_all_activation_layers(model)

intermediate_act_fn_0
intermediate_act_fn_1
intermediate_act_fn_2
intermediate_act_fn_3
intermediate_act_fn_4
intermediate_act_fn_5
intermediate_act_fn_6
intermediate_act_fn_7
intermediate_act_fn_8
intermediate_act_fn_9
intermediate_act_fn_10
intermediate_act_fn_11
activation
2


In [9]:
# Run Model
outputs = model(**encoding)